In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from dateutil.relativedelta import relativedelta
import sys 
sys.path.append('../maths/') 
from baseball_stats import BasicHitting


In [2]:
k = ['playerID', 'birthYear', 'birthMonth', 'birthDay']
import_people = pd.read_csv('../datafiles/People.csv', encoding='latin-1', usecols=k)
import_batting = pd.read_csv('../datafiles/Batting.csv', encoding='latin-1')
people = import_people.copy()

#fix fucky dates
people['birthYear'] = people['birthYear'].fillna(1875).astype(int)
people['birthMonth'] = people['birthMonth'].fillna(1).astype(int)
people['birthDay'] = people['birthDay'].fillna(1).astype(int)

#create a birthdate column so we can calulate an age at start of season
people['birthdate'] = people.apply(lambda x: f"""{x['birthYear']}-{x['birthMonth']}-{x['birthDay']}""", axis=1)
people['birthdate'] = pd.to_datetime(people['birthdate'], errors='coerce')

#merge the people and batting dataframes
batting = people.copy().merge(import_batting, on='playerID', how ='inner')

#make sure the raw stats are 0 and not null
batting.loc[:, ['G', 'AB', 'R', 'H', '2B', '3B', 'HR', 'RBI', 'SB', 'CS', 'BB', 'SO', 'IBB', 'HBP', 'SH', 'SF', 'GIDP']].fillna(0, inplace=True)

In [ ]:
#TODO: create as function to do career and either 162 game or 600 pa stats
player_batting = batting.copy()
player_batting = player_batting.drop(columns=['stint', 'teamID', 'lgID', 'birthYear', 'birthMonth', 'birthDay', 'G_old', 'G_batting'], axis=1).groupby(['playerID', 'birthdate', 'yearID']).sum().reset_index()
#estimate the start of the season at april 1
player_batting['season_start'] = player_batting.apply(lambda x: f"""{x['yearID']}-04-01""", axis=1)
player_batting['season_start'] = pd.to_datetime(player_batting['season_start'], errors='coerce')

#calculate the age of the player at the start of the season
player_batting['age'] = player_batting.apply(lambda x: relativedelta(x['season_start'], x['birthdate']).years, axis=1)
player_batting.drop(columns='birthdate', inplace=True)
player_batting['rowType'] = 'season'
player_batting.rename(columns={'3B':'B3', '2B':'B2'}, inplace=True)

#adding shit up
player_batting['PA'] = BasicHitting.calc_pa(player_batting)
player_batting['B1'] = BasicHitting.calc_b1(player_batting)
player_batting['TB'] = BasicHitting.calc_tb(player_batting)
player_batting['SBA'] = BasicHitting.calc_sba(player_batting)
player_batting['BA'] = BasicHitting.calc_ba(player_batting)
player_batting['XBH'] = BasicHitting.calc_xbh(player_batting)
player_batting['TOB'] = BasicHitting.calc_tob(player_batting)
player_batting['RP'] = BasicHitting.calc_rp(player_batting)

#create rows for career
career_df = player_batting.copy()


,playerID,birthdate,yearID,G,AB,R,H,B2,B3,HR,...,EQA,ISO,GPA,PASO,BBK,RC,BR,SBP,TA,BABIP
0,aardsda01,1981-12-27,2004,11,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN
1,aardsda01,1981-12-27,2006,45,2,0,0,0,0,0,...,0.333,0.000,NaN,NaN,NaN,-0.168800,NaN,NaN,0.000000,0.000
2,aardsda01,1981-12-27,2007,25,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN
3,aardsda01,1981-12-27,2008,47,1,0,0,0,0,0,...,0.000,0.000,NaN,1.000,0.000,-0.110667,NaN,NaN,0.000000,NaN
4,aardsda01,1981-12-27,2009,73,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105145,zwilldu01,1888-11-02,1915,150,548,65,157,32,7,13,...,0.847,0.156,0.2707,9.769,1.031,101.563780,93.227,1.0,0.856777,0.306
105146,zwilldu01,1888-11-02,1916,35,53,4,6,1,0,1,...,0.407,0.076,0.1233,9.833,0.667,0.951824,2.281,NaN,0.297872,0.109
105147,zychto01,1990-08-07,2015,13,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN
105148,zychto01,1990-08-07,2016,12,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN


In [ ]:

    
#ratios and percentages
player_batting['HRAB'] = BasicHitting.calc_hrab(player_batting)
player_batting['HRH'] = BasicHitting.calc_hrh(player_batting)
player_batting['OBP'] = BasicHitting.calc_obp(player_batting)
player_batting['SLGp'] = BasicHitting.calc_slg(player_batting)
player_batting['OPS'] = BasicHitting.calc_ops(player_batting)
player_batting['XBHp'] = BasicHitting.calc_xbh_pct(player_batting)
player_batting['EqA'] = BasicHitting.calc_eqa(player_batting)
player_batting['ISO'] = BasicHitting.calc_iso(player_batting)
player_batting['GPA'] = BasicHitting.calc_gpa(player_batting)
player_batting['PASO'] = BasicHitting.calc_paso(player_batting)
player_batting['BBK'] = BasicHitting.calc_bbk(player_batting)
player_batting['RC'] = BasicHitting.calc_rc(player_batting)
player_batting['BR'] = BasicHitting.calc_br(player_batting)
player_batting['SBp'] = BasicHitting.calc_sbp(player_batting)
player_batting['TA'] = BasicHitting.calc_ta(player_batting)
player_batting['BABIP'] = BasicHitting.calc_babip(player_batting)

player_batting

In [ ]:
#TODO: combine and export to csv